In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} {utils_path} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


import regrid_gwana_wrt as gwa


"""
#############
# The old ways ....
#########################################
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')
sys.path.append(workdir_ + 'myPythonTools/CASutils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

import regrid_gwana_latSlice_wrt as gwa

# Own local packages
import AveragingUtils as Av
import PlotUtil as Pu
import utils as uti
import numerical_utils as nuti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL
import filter_utils as fu

sys.path.append(workdir_ + 'PyRegridding/Drivers/')
import RegridField as RgF
"""

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime
import yaml
#from box import Box

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(GrU)
importlib.reload(gwa)


In [ ]:
gwa.driver()

In [ ]:


with open('configure_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)


######################################################

x='ne240x2_QxQsst'
#x='ne240x2'
#x='oldCTL' #'waccmL135'
exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
ymdPat = cfg[x]['ymdPat']
hsPat = cfg[x]['hsPat']
print( exp, subd, Src, Hkey, Dst, useri )

year,month,day,hour = 1985,1,1,0
second = hour  * 3600
ymdPat=str( year ).zfill(4) + '-' + str(month).zfill(2) + '-' + str(day).zfill(2) + '-' + str(second).zfill(5)

ymdPat='2004-07-01-*'
A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat=hsPat , ymdPat=ymdPat,verbose=True, open_dataset=True )



In [ ]:

#target_plevs = [ 931.99204257 855.78421029 498.51109676 227.21837597  58.48490616, 2.80419527]

target_plevs = [ 932.0, 856., 499., 227.0,  58., 2.8]
plevs = A.X.lev.values

#print( A.X.lev.values[np.array([85, 80, 68, 57,38,11 ] )]) 
print(plevs)


In [ ]:
lev_sel = uti.find_nearest_plev_indices(plev=plevs , target_levels=target_plevs )
print(lev_sel)

In [ ]:
A.X = A.X.isel( lev=lev_sel )

In [ ]:
A.X

In [ ]:
print(A.X.TS.dims)
print( ('time', 'ncol'))
print(  A.X.TS.dims == ('time', 'lev','ncol'))
print(  A.X.TS.dims == ('time',   'ncol',))
print(  A.X.TS.dims == ('time',))


In [ ]:
A.X['U'].long_name

In [ ]:
%%time
##########################################################
# Generate regirdding objetcs to be usedf later on
#---------------------------------------------------------

RgOb_ne240_x_ne16 = RgF.Horz( Src='ne240pg3', Dst='ne16pg3' ) 

RgOb_ne16_x_ne240 = RgF.Horz(  Src='ne16pg3' , Dst='ne240pg3' , RegridMethod='BILINEAR' ) 

RgOb_ne240_x_llOxO = RgF.Horz( Src='ne240pg3', Dst='latlonOxO' ) 


In [ ]:
Topo=xr.open_dataset(A.X.topography_file)
topoO=Topo.PHIS.values / 9.8

In [ ]:
ogO=A.X.OMEGA.values

print(ogO.shape)

ogOx2=RgF.Horz(xfld_Src=ogO , Src='ne240pg3', Dst='ne16pg3' , RegridObj_In=  RgOb_ne240_x_ne16  ) 

ogOx2xO=RgF.Horz(xfld_Src=ogOx2 , Src='ne16pg3' , Dst='ne240pg3', RegridObj_In= RgOb_ne16_x_ne240  ) 

latO,lonO = GrU.latlon( grid='ne240pg3' )

print( ogO.shape )
print( ogOx2.shape )
print( ogOx2xO.shape )



In [ ]:
lev=A.X.lev.values
lat2,lon2 = GrU.latlon( grid='ne16pg3' )

In [ ]:
dims   = ["ncol","ncol2","time","lev"]
coords = dict( 
    lon  = ( ["ncol"],lonO),
    lat  = ( ["ncol"],latO ),
    lon2  = ( ["ncol2"],lon2),
    lat2  = ( ["ncol2"],lat2 ),
    lev  = ( ["lev"],lev),
    time = ( ["time"], A.X.time.values  ), #pd.to_datetime( pdTime_ERA[itim] ) ),
)

Xw = xr.Dataset( coords=coords  )
Dar = xr.DataArray( data=topoO, 
                    dims=('ncol',),
                    attrs=dict( description='Topography',units='m',) ,) 
Xw['topoO'] = Dar
Dar = xr.DataArray( data=ogO, 
                    dims=('time','lev','ncol',),
                    attrs=dict( description='Pressure velocity',units='Pa s-1',) ,) 
Xw['omegaO'] = Dar

Dar = xr.DataArray( data=ogOx2, 
                    dims=('time','lev','ncol2',),
                    attrs=dict( description='Pressure velocity',units='Pa s-1',) ,) 
Xw['omegaOx2'] = Dar

Dar = xr.DataArray( data=ogOx2xO, 
                    dims=('time','lev','ncol',),
                    attrs=dict( description='Pressure velocity',units='Pa s-1',) ,) 
Xw['omegaOx2xO'] = Dar


In [ ]:
###
Xw

In [ ]:
###
import os
tootrange = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"
fname=f'{A.exp}_GWana_{tootrange}.nc'
Bdiro=f'/glade/derecho/scratch/juliob/archive/{A.exp}/atm/GWana/'
#######

fname = f'{Bdiro}{fname}'
print(Bdiro)
print(fname)

In [ ]:
os.makedirs( Bdiro , exist_ok=True )
Xw.to_netcdf( fname )

In [ ]:
#A.X.area
print( np.sum(A.X.area.values ) , 4.*np.pi )

In [ ]:
#A.X.area

print( np.sum( A.X.area.values*ogOx2xO[0,:] ) , 4.*np.pi )

#plt.plot( ogOx2xO[0,:] )

In [ ]:
dog = ogO - ogOx2xO

In [ ]:
topoll =RgF.Horz(xfld_Src=topoO , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO  ) 


In [ ]:
dogll =RgF.Horz(xfld_Src=dog , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO  ) 


In [ ]:
ogOll =RgF.Horz(xfld_Src=ogO , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO  ) 
ogOx2xOll =RgF.Horz(xfld_Src=ogOx2xO , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO  ) 


In [ ]:

dolev = np.linspace(-1,1,num=21)
olev = np.linspace(-10,10,num=21)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()


k=4
co=ax[0].contourf( ogOll[0,k,:,:] , levels = 0.5*olev, cmap='bwr' )
ax[0].set_xlim((500,1000))
ax[0].set_ylim((100,400))
cb=plt.colorbar( co )
co=ax[1].contourf( ogOx2xOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
ax[1].set_xlim((500,1000))
ax[1].set_ylim((100,400))
cb=plt.colorbar( co )
co=ax[2].contourf( dogll[0,k,:,:] , levels = 2*dolev , cmap='bwr' )
ax[2].set_xlim((500,1000))
ax[2].set_ylim((100,400))
cb=plt.colorbar( co )
#plt.colorbar()

In [ ]:
toot = (A.X.time[0].values.item() ).strftime('%Y-%m-%d:%HZ')
print( toot )

In [ ]:
A.exp

In [ ]:
plev=A.X.lev.values
#print(plev)
zlev= -7.0 * np.log( plev / 1000. )
print(plev)

In [ ]:
t=14*4
toot = (A.X.time[t].values.item() ).strftime('%Y-%m-%d:%HZ')
print(toot)

In [ ]:


lats,lons = GrU.latlon(grid='latlonOxO')





In [ ]:

fig, ax = plt.subplots(2, 3, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

dolev = np.linspace(-1,1,num=21)
olev = np.linspace(-10,10,num=21)

jj = 300

tootrange = f" {(A.X.time[0].values.item() ).strftime('%Y-%m-%d')} to {(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"
#loot=f"Lat={A.X.lat[jj].values:.1f}"
latval = lats[jj] 
print(f" Lat={latval:.1f} ")
print(tootrange )

klist=np.arange( len(zlev) )
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.25
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf(scak* dogll[:,k,jj,:],levels=2*dolev ,cmap='bwr' )
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    cb=plt.colorbar( co )
    p=p+1

fig.suptitle( f'{A.exp}, {tootrange}, Lat={latval:.1f}' , fontsize=20)

FigName=f'Hpass-omega-Hovmueller_{A.exp}.png'
Pu.save_link(png_filename = FigName )


In [ ]:
dogsq = dogll*dogll

In [ ]:
doggee=np.sqrt( np.average( dogsq, axis=0 ) )

In [ ]:


dolev = np.linspace(0,1,num=21)
olev = np.linspace(-10,10,num=21)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(2, 3, figsize=(28, 10))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

xlim=np.array([0,2880])
ylim=np.array([0,1440])

t=13*4
toot = (A.X.time[t].values.item() ).strftime('%Y-%m-%d:%HZ')
tootrange = f" {(A.X.time[0].values.item() ).strftime('%Y-%m-%d')} to {(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"

klist=np.arange( len(zlev) ) #[90,84, 80,75, 68, 65, 57 , 50 ,38,30,20,10 ]
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.25
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf( scak*doggee[k,:,:] , levels = dolev , cmap='gist_ncar' )
    co2 = ax[p].contour( topoll , levels = [1,10,1000], colors='black' )
    #ax[p].set_xlim((500,1000))
    #ax[p].set_ylim((100,400))
    ax[p].set_xlim(xlim)
    ax[p].set_ylim(ylim)
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    cb=plt.colorbar( co )
    p=p+1
fig.suptitle( f'{A.exp}, {tootrange}' , fontsize=20)
#plt.colorbar()
FigName=f'Hpass-omega-rms_{A.exp}.png'
Pu.save_link(png_filename = FigName )


In [ ]:
plt.plot( np.average( doggee[5,:,:], axis=1 ) )

In [ ]:


dolev = np.linspace(-1,1,num=21)
olev = np.linspace(-10,10,num=21)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(3, 4, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

xlim=np.array([200,1300])-200
ylim=np.array([100,700])

t=13*4
toot = (A.X.time[t].values.item() ).strftime('%Y-%m-%d:%HZ')
klist=np.arange( len(zlev) ) #[90,84, 80,75, 68, 65, 57 , 50 ,38,30,20,10 ]
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf( scak*dogll[t,k,:,:] , levels = 2*dolev , cmap='bwr' )
    co2 = ax[p].contour( topoll , levels = [1,10,1000], colors='black' )
    #ax[p].set_xlim((500,1000))
    #ax[p].set_ylim((100,400))
    ax[p].set_xlim(xlim)
    ax[p].set_ylim(ylim)
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    cb=plt.colorbar( co )
    p=p+1
fig.suptitle( f'{A.exp}, time={toot}' , fontsize=20)
#plt.colorbar()
FigName=f'Hpass-omega.png'
Pu.save_link(png_filename = FigName )


In [ ]:


dolev = np.linspace(-1,1,num=21)
olev = np.linspace(-10,10,num=21)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(3, 4, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

t=0
toot = (A.X.time[t].values.item() ).strftime('%Y-%m-%d:%HZ')
klist=[90,85, 80,75, 70, 65, 60 , 50 ,40,30,20,10 ]
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    co=ax[p].contourf( scak*ogOll[t,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co2 = ax[p].contour( topoll , levels = [1,10,1000], colors='black' )
    #ax[p].set_xlim((500,1000))
    #ax[p].set_ylim((100,400))
    ax[p].set_xlim((200,1300))
    ax[p].set_ylim((100,700))
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    cb=plt.colorbar( co )
    p=p+1
fig.suptitle( f'{A.exp}, time={toot}' , fontsize=20)
#plt.colorbar()
FigName=f'Raw-omega.png'
Pu.save_link(png_filename = FigName )


In [ ]:
###

In [ ]:
###

In [ ]:
###
poo=1

In [ ]:
###
xoo=poo